In [ ]:
# !pip install spotipy

In [51]:
import operator
import sys
import pyspark
from pyspark.sql import SparkSession, types
from pyspark import SparkConf, SparkContext
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [52]:
CLIENT_ID = 'd3e048fac0e8438cab168ac60459044c'
CLIENT_SECRET = '4d47925e9199481fb41cc0c1406de938'

In [53]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
cid = CLIENT_ID
secret = CLIENT_SECRET
client_credentials_manager = SpotifyClientCredentials(
    client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [54]:
# create lists of columns to be used when reading/merging the csv's
columns = ['title', 'rank', 'date', 'artist', 'url', 'region', 'chart', 'trend', 'streams']
DATA_PATH = "gs://big_data_spotify_bucket/charts.csv"
# Spark settings
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)
# sp.audio_features(track_url)[0]

In [55]:
# Load the data, cache this since we're accessing this each iteration
data = spark.read.format("csv").option("header", "true").load(DATA_PATH).cache()
#===============================

22/12/17 20:36:46 WARN org.apache.spark.sql.execution.CacheManager: Asked to cache already cached data.


In [56]:
data = data.withColumn("rank", data["rank"].cast("int")).withColumn("streams", data["streams"].cast("int"))
data.printSchema()
data.show(20)
# data.count()

root
 |-- title: string (nullable = true)
 |-- rank: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- artist: string (nullable = true)
 |-- url: string (nullable = true)
 |-- region: string (nullable = true)
 |-- chart: string (nullable = true)
 |-- trend: string (nullable = true)
 |-- streams: integer (nullable = true)

+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|               title|rank|      date|              artist|                 url|   region| chart|        trend|streams|
+--------------------+----+----------+--------------------+--------------------+---------+------+-------------+-------+
|Chantaje (feat. M...|   1|2017-01-01|             Shakira|https://open.spot...|Argentina|top200|SAME_POSITION| 253019|
|Vente Pa' Ca (fea...|   2|2017-01-01|        Ricky Martin|https://open.spot...|Argentina|top200|      MOVE_UP| 223988|
|Reggaetón Lento (...|   3|2017-01-01|                CNCO|

In [57]:
data3 = pd.read_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")
# data.take(1)
url_column = data.select('url')
# print(url_column.take(1)[0][0]) 
print(data3.iloc[99999])
fet = sp.audio_features(data3.iloc[99999, 5])[0]
# fet
cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
for val in cols_to_drop2:
    del fet[val]
print(fet)
data3.iloc[99999, 10:] = fet.values()
data3.iloc[99999]
# 7w87IxuO7BDcJ3YUqCyMTT

Unnamed: 0                                                      99999
title                                                Treat You Better
rank                                                               20
date                                                       2017-01-04
artist                                                   Shawn Mendes
url                 https://open.spotify.com/track/4Hf7WnR761jpxPr...
region                                                       Portugal
chart                                                          top200
trend                                                         MOVE_UP
streams                                                       10763.0
danceability                                                      NaN
energy                                                            NaN
loudness                                                          NaN
speechiness                                                       NaN
acousticness        

Unnamed: 0                                                      99999
title                                                Treat You Better
rank                                                               20
date                                                       2017-01-04
artist                                                   Shawn Mendes
url                 https://open.spotify.com/track/4Hf7WnR761jpxPr...
region                                                       Portugal
chart                                                          top200
trend                                                         MOVE_UP
streams                                                       10763.0
danceability                                                    0.624
energy                                                          0.803
loudness                                                       -4.105
speechiness                                                     0.246
acousticness        

In [58]:
# fet = fet['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'id', 'uri', 'duration_ms', 'time_signature']


In [59]:
data_pd = data.limit(100000).toPandas()
data_pd.shape

(100000, 9)

In [60]:
def convert_to_pd_and_add_features():
    track_features = []
    tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
    cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
    for index, row in data_pd.iterrows():
        if index % 1000 == 0:
            print(index)
    #         print(sp.audio_features(row['url'])[0])  
            temp_df = tf_df.drop(columns=cols_to_drop2)
            new_df = pd.concat([data_pd, temp_df], axis=1)
    #         print(new_df.shape)
            new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")

        audio_features = sp.audio_features(row['url'])[0]
        tf_df = tf_df.append(audio_features, ignore_index = True)

    #     track_features.append(audio_features)

    # tf_df.head()
    temp_df = tf_df.drop(columns=cols_to_drop2)
    new_df = pd.concat([data_pd, temp_df], axis=1)
    print(new_df.shape)
    new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")

In [61]:
# cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
# tf_df = tf_df.drop(columns=cols_to_drop2)
# print(track_df.info())
# print(tf_df.info())
# new_df = pd.concat([data_pd, tf_df], axis=1)
# new_df.head()
# sparkDF=spark.createDataFrame(new_df) 
# sparkDF.printSchema()
# sparkDF.show(20)

In [62]:
# new_df.shape

In [63]:
# save data
# new_df.to_csv("gs://big_data_spotify_bucket/processed_data.csv")

In [64]:
# # Define the number of splits you want
# n_splits = 6
  
# # Calculate count of each dataframe rows
# each_len = 26173514 // n_splits
  
# # Create a copy of original dataframe
# copy_df = data
  
# # Iterate for each dataframe
# i = 0
# while i < n_splits:
  
#     # Get the top `each_len` number of rows
#     temp_df = copy_df.limit(each_len)
  
#     # Truncate the `copy_df` to remove
#     # the contents fetched for `temp_df`
#     copy_df = copy_df.subtract(temp_df)
  
#     # View the dataframe
# #     print(temp_df.count())
# #     temp_df.show(20)
#     temp = temp_df.toPandas()
#     print(temp.shape)
#     # Increment the split number
#     i += 1

In [65]:
sp.audio_features('7w87IxuO7BDcJ3YUqCyMTT')[0]

{'danceability': 0.733,
 'energy': 0.71,
 'key': 5,
 'loudness': -5.849,
 'mode': 0,
 'speechiness': 0.0292,
 'acousticness': 0.145,
 'instrumentalness': 0.115,
 'liveness': 0.0956,
 'valence': 0.965,
 'tempo': 127.975,
 'type': 'audio_features',
 'id': '7w87IxuO7BDcJ3YUqCyMTT',
 'uri': 'spotify:track:7w87IxuO7BDcJ3YUqCyMTT',
 'track_href': 'https://api.spotify.com/v1/tracks/7w87IxuO7BDcJ3YUqCyMTT',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7w87IxuO7BDcJ3YUqCyMTT',
 'duration_ms': 239600,
 'time_signature': 4}

In [89]:
data1 = pd.read_csv("gs://big_data_spotify_bucket/processed_data.csv")
data2 = pd.read_csv("gs://big_data_spotify_bucket/processed_data_v2.csv")
data3 = pd.read_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")

In [90]:
# new_df = pd.concat([data1, data2], axis=0)
# # new_df.to_csv("gs://big_data_spotify_bucket/processed_data_v3.csv")
# new_df2 = new_df.iloc[0:100000]
# new_df2
data1.drop(columns=data1.columns[0], axis=1, inplace=True)
data2.drop(columns=data2.columns[0], axis=1, inplace=True)
data3.drop(columns=data3.columns[0], axis=1, inplace=True)

In [91]:
data1.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        70006
energy              70006
loudness            70006
speechiness         70006
acousticness        70006
instrumentalness    70006
liveness            70006
valence             70006
tempo               70006
id                  70006
uri                 70006
duration_ms         70006
time_signature      70006
dtype: int64

In [69]:
data2.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        30014
energy              30014
loudness            30014
speechiness         30014
acousticness        30014
instrumentalness    30014
liveness            30014
valence             30014
tempo               30014
id                  30014
uri                 30014
duration_ms         30014
time_signature      30014
dtype: int64

In [70]:
data3.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11142
danceability        16017
energy              16017
loudness            16017
speechiness         16017
acousticness        16017
instrumentalness    16017
liveness            16017
valence             16017
tempo               16017
id                  16017
uri                 16017
duration_ms         16017
time_signature      16017
dtype: int64

In [108]:
columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
cols_to_drop2 = ['key', 'mode', 'type', 'track_href', 'analysis_url']
def fill_in_missing_data():
    for i in range(len(data3.index)) :
        if data3.iloc[i].isnull().sum() > 1:
            print("Nan in row ", i , " : " ,  data3.iloc[i].isnull().sum())
            audio_features = sp.audio_features(data3.iloc[i, 4])[0]
    #         print(audio_features)
            for val in cols_to_drop2:
                del audio_features[val]
    #         print(data3.iloc[i])
            data3.iloc[i, 9:] = audio_features.values()
    #         print(data3.iloc[i])
fill_in_missing_data()
# print(data3.iloc[89751])
# print()
# print(data3.iloc[89752])
# print()
# print(data3.iloc[89753])

In [109]:
data3.isnull().sum()

title                   0
rank                    0
date                    0
artist                  0
url                     0
region                  0
chart                   0
trend                   0
streams             11139
danceability            0
energy                  0
loudness                0
speechiness             0
acousticness            0
instrumentalness        0
liveness                0
valence                 0
tempo                   0
id                      0
uri                     0
duration_ms             0
time_signature          0
dtype: int64

In [107]:
temp = data3
temp
tem.drop(columns=data3.columns[8], axis=1, inplace=True)

,title,rank,date,artist,url,region,chart,trend,streams,danceability,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,time_signature
0,Chantaje (feat. Maluma),1,2017-01-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,253019.0,0.852,...,0.0776,0.18700,0.000030,0.1590,0.907,102.034,6mICuAdrwEjh6Y6lroV2Kg,spotify:track:6mICuAdrwEjh6Y6lroV2Kg,195840.0,4.0
1,Vente Pa' Ca (feat. Maluma),2,2017-01-01,Ricky Martin,https://open.spotify.com/track/7DM4BPaS7uofFul...,Argentina,top200,MOVE_UP,223988.0,0.663,...,0.2260,0.00431,0.000017,0.1010,0.533,99.935,7DM4BPaS7uofFul3ywMe46,spotify:track:7DM4BPaS7uofFul3ywMe46,259196.0,4.0
2,Reggaetón Lento (Bailemos),3,2017-01-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,MOVE_DOWN,210943.0,0.761,...,0.0502,0.40000,0.000000,0.1760,0.710,93.974,3AEZUABDXNtecAOSC1qTfo,spotify:track:3AEZUABDXNtecAOSC1qTfo,222560.0,4.0
3,Safari,4,2017-01-01,"J Balvin, Pharrell Williams, BIA, Sky",https://open.spotify.com/track/6rQSrBHf7HlZjtc...,Argentina,top200,SAME_POSITION,173865.0,0.508,...,0.3260,0.55100,0.000003,0.1260,0.555,180.044,6rQSrBHf7HlZjtcMZ4S4bO,spotify:track:6rQSrBHf7HlZjtcMZ4S4bO,205600.0,4.0
4,Shaky Shaky,5,2017-01-01,Daddy Yankee,https://open.spotify.com/track/58IL315gMSTD37D...,Argentina,top200,MOVE_UP,153956.0,0.899,...,0.2920,0.07600,0.000000,0.0631,0.873,88.007,58IL315gMSTD37DOZPJ2hf,spotify:track:58IL315gMSTD37DOZPJ2hf,234320.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Black Beatles,16,2017-01-04,"Rae Sremmurd, Gucci Mane",https://open.spotify.com/track/6fujklziTHa8uoM...,Portugal,top200,SAME_POSITION,12072.0,0.794,...,0.0649,0.14200,0.000000,0.1280,0.355,145.926,6fujklziTHa8uoM5OQSfIo,spotify:track:6fujklziTHa8uoM5OQSfIo,291893.0,4.0
99996,Side To Side,17,2017-01-04,"Ariana Grande, Nicki Minaj",https://open.spotify.com/track/4pLwZjInHj3SimI...,Portugal,top200,SAME_POSITION,11790.0,0.648,...,0.2470,0.04080,0.000000,0.2920,0.603,159.145,4pLwZjInHj3SimIyN9SnOz,spotify:track:4pLwZjInHj3SimIyN9SnOz,226160.0,4.0
99997,Scars To Your Beautiful,18,2017-01-04,Alessia Cara,https://open.spotify.com/track/0prNGof3XqfTvND...,Portugal,top200,MOVE_UP,11620.0,0.573,...,0.1290,0.02850,0.000000,0.1110,0.451,97.085,0prNGof3XqfTvNDxHonvdK,spotify:track:0prNGof3XqfTvNDxHonvdK,230227.0,4.0
99998,In the Name of Love,19,2017-01-04,"Martin Garrix, Bebe Rexha",https://open.spotify.com/track/04DwTuZ2VBdJCCC...,Portugal,top200,MOVE_DOWN,11550.0,0.490,...,0.0406,0.05920,0.000000,0.3370,0.196,133.889,04DwTuZ2VBdJCCC5TROn7L,spotify:track:04DwTuZ2VBdJCCC5TROn7L,195840.0,4.0


In [ ]:
data3.isnull().sum()

In [ ]:
data3